# Logistic regression with Professor Mittens, a.k.a. vanilla, multinomial and ordinal logistic regression.

## Overview

In this notebook we will learn how to use logistic regression to study the factors that affect the opinions cats hold regarding lockdowns and vaccination as methods to control the spread of SARS-CoV-2. This will start with a visual inspection of the data, which is survey data of a population of cats, followed by the development of three different logistic models to explain the data.

## Types of logistic regression

Logistic regression typically refers to binary classification, multinomial logistic regression refers to classification where there are multiple values without an ordering associated with them and ordinal logistic regression refers to the case where there are multiple ordered values that the catagorical variable may take.

## Survey data

Professor Mittens in interested in helping the government to understand the opinions the cat public holds regarding the use of lockdown and vaccines against SARS-CoV-2. To learn more about this, he has interviewed 1000 cats. The data in `cat-opinions.csv` contains measurments of the following:

- `whisker_length` is the length of their whiskers in centimeters,
- `work_from_home` is a Boolean variable indicating whether the cat can work from home, which is 1 if they can work from home and 0 if not.
- `trust_in_government` is a value from 0 to 100 indicating the level of trust put in the government,
- `fifth_generation` is a Boolean variable indicating whether the cat thinks 5G is a government conspiracy, which is 1 if they think 5G is a conspiracy and 0 if not.
- `support_lockdown` is a Likert scale response about whether you support the lockdown measures, this is coded from "strongly against", "against", "neutral", "support", "strongly support" as 0--5.
- `will_vaccinate` is a `Maybe Bool` indicating if the cat will accept a vaccination, this is coded as follows: "yes" as 1, "no answer" as 0 and "no" as -1.

It is the variables `support_lockdown` and `will_vaccinate` that the goverment is interested in understanding so they can design a suitable response.

## Questions to answer

Professor Mittens is interested in answering the question

- Will herd immunity be reached with vaccination?

Of course, we need to formalise this before they can be answered properly. In the case of herd immunity, we are trying to estimate the proportion of cats that will agrree to be vaccinated. For simplicity, we will ignore that some cats cannot be vaccinated for various reasons. Let's assume that you need to vaccinate 85% of the population to get herd immunity. Does this seem likely to happen?

In [1]:
%matplotlib inline
from typing import List, Any, Tuple
from functools import reduce
from itertools import repeat
import math as math

import pandas as pd
import numpy as np
import altair as alt
import scipy.stats as stats

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.genmod import families
from statsmodels.stats import descriptivestats

In [2]:
data_df = pd.read_csv("cat-opinions.csv")

## Exploratory data analysis

Start by inspecting the data contained in `cat-opinions.csv` to see if there is anything that might be useful for predicting a cat's opinions of lockdown and vaccination. For example:

- consider `head`, `describe` and `corr`
- consider a contingency table [contingency table](https://www.statsmodels.org/stable/contingency_tables.html) after [cross tabulating](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html)

In [3]:
data_df.head(5)

,work_from_home,whisker_length,trust_in_government,fifth_generation,will_vaccinate,support_lockdown
0,0,7.844678,60.151674,0,0,0
1,1,8.436025,55.094887,0,1,0
2,0,8.011435,57.583511,0,1,3
3,1,8.101098,59.243290,0,1,4
4,0,8.965462,54.865060,0,1,4


In [4]:
data_df.describe()

,work_from_home,whisker_length,trust_in_government,fifth_generation,will_vaccinate,support_lockdown
count,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
mean,0.39800,8.163011,63.352821,0.094000,0.719000,2.92400
std,0.48973,0.483593,7.198005,0.291975,0.601998,1.68792
min,0.00000,6.758440,50.721931,0.000000,-1.000000,0.00000
25%,0.00000,7.833074,56.893063,0.000000,1.000000,1.00000
50%,0.00000,8.136991,63.627789,0.000000,1.000000,4.00000
75%,1.00000,8.502563,69.559004,0.000000,1.000000,4.00000
max,1.00000,9.761475,75.676853,1.000000,1.000000,4.00000


In [5]:
data_df.corr()

,work_from_home,whisker_length,trust_in_government,fifth_generation,will_vaccinate,support_lockdown
work_from_home,1.000000,-0.022976,0.006148,0.025118,0.060566,0.064481
whisker_length,-0.022976,1.000000,0.044197,-0.050090,0.049822,-0.003602
trust_in_government,0.006148,0.044197,1.000000,-0.175242,0.293528,0.316420
fifth_generation,0.025118,-0.050090,-0.175242,1.000000,-0.384903,-0.180478
will_vaccinate,0.060566,0.049822,0.293528,-0.384903,1.000000,0.143476
support_lockdown,0.064481,-0.003602,0.316420,-0.180478,0.143476,1.000000


In [6]:
my_cross_tab = pd.crosstab(data_df["work_from_home"], data_df["will_vaccinate"])
my_contingency_table = sm.stats.Table(my_cross_tab)
print(my_contingency_table)
my_rslt = my_contingency_table.test_nominal_association()
my_rslt.pvalue

A 2x3 contingency table with counts:
[[ 58.  71. 473.]
 [ 22.  50. 326.]]


0.0638421236629041

In [7]:
my_cross_tab = pd.crosstab(data_df.fifth_generation, data_df.will_vaccinate)
my_contingency_table = sm.stats.Table(my_cross_tab)
print(my_contingency_table)
my_rslt = my_contingency_table.test_nominal_association()
my_rslt.pvalue

A 2x3 contingency table with counts:
[[ 42. 103. 761.]
 [ 38.  18.  38.]]


0.0

In [8]:
whisker_boxplot = (alt
               .Chart(data_df)
               .mark_boxplot()
               .encode(y = alt.Y("whisker_length",
                                 scale=alt.Scale(zero=False)),
                       x = "will_vaccinate:O")
                  )

trust_boxplot = (alt
                .Chart(data_df)
                .mark_boxplot()
                .encode(y = alt.Y("trust_in_government",
                                 scale = alt.Scale(zero = False)),
                       x = "will_vaccinate:O"))

whisker_boxplot | trust_boxplot

alt.HConcatChart(...)

### Question: What proportion of cats will accept a vaccine?

The following definition from Example 1 might be useful here...

In [9]:
CI = Tuple[float,float]
EstimateAndCI = Tuple[float,CI]

def wald_estimate_and_ci(num_trials: int, num_success: int) -> EstimateAndCI:
    p_hat = num_success / num_trials
    z = 1.96
    delta = z * math.sqrt(p_hat * (1 - p_hat) / num_trials)
    return (p_hat,(p_hat - delta, p_hat + delta))

## Logistic regression to predict vaccination

Use logistic regression study cats' reponses to the vaccination question among those who provided an answer. 

#### Variable encoding

Note that the following code throws an error!

```
ternary_df = data_df.copy()
ternary_df = ternary_df[data_df.will_vaccinate != 0]

formula = "will_vaccinate ~ C(work_from_home) + whisker_length + trust_in_government + C(fifth_generation)"
ternary_logistic = smf.logit(formula = formula,
                             data = ternary_df).fit()
```

It will throw a value error: `ValueError: endog must be in the unit interval.`.

### Question: Did the optimisation converge? Why does this matter now?

### Question: What does the coefficient of the working from home variable and the level of trust in the goverment mean?

# WARNING!!!

Please keep in mind that this is a factor change to the odds (which is $p/(1-p)$ recall) so it is non-trivial to interpret how such a change will alter the probability of each outcome. Interpreting a table of coefficients is difficult, even when you convert it to multiplicative changes to the odds since there are non-linearities involved. A nicer way to do this is with _effect displays_ which is something that Fox has written software to generate. We will see these later in this tutorial.

### Question: What proportion of those that did not answer will accept a vaccine?

## Multinomial logistic regression for vaccination

Use multinomial logistic regression to study the cat's response to the vaccination question

### Question: Why do we now have two sets of parameters?

## Jumping to R

The interrogation of these models will be much easier using some of the packages available for R, so without further ado we shall re-fit this model in R.